In [11]:
import pickle
import json
import torch
from mlflux.predictor import FluxANNs

def open_case (model_dir, model_name):
    with open(model_dir + 'config.json', 'r') as f:
        config = json.load(f)   
    filename = model_dir + model_name
    with open(filename, "rb") as input_file:
        model = pickle.load(input_file)   
    model.config = config 
    return model

########## The old way of loading model class through pickles ##########
# source_path = '/scratch/jw8736/mlflux/example_python/'
# M = open_case(source_path + 'M/', 'm.p')
# SH = open_case(source_path + 'SH/', 'sh.p')
# LH = open_case(source_path + 'LH/', 'lh.p')
# M.__class__
# M.__class__.__module__

### Convert previously saved pickle to PyTorch-native binary format

In [ ]:
import os 
source_path = '/scratch/jw8736/mlflux/example_python/'
target_path = '/scratch/jw8736/mlflux/example_torch_script/'

# Apparently change M/SH/LH
old_model = open_case(source_path + 'LH/', 'lh.p') # this was a class not good for torch script
dir_name = target_path + 'LH/'
os.makedirs(dir_name, exist_ok=True)

# Don't use this that save scale and weights seperately, use the one that saves scale and weights together
# torch.save(LH.Xscale, dir_name + "Xscale.pt")
# torch.save(LH.Yscale, dir_name + "Yscale.pt")
# torch.save(LH.mean_func.state_dict(), dir_name + "mean_weights.pt")
# torch.save(LH.var_func.state_dict(), dir_name + "var_weights.pt")

In [3]:
from flux_model import ANN_online

####### Convert mean network and save #######
# Hard code configuration for now 
n_in = 5
n_out = 1
hidden_channels = [32, 16] 
activation = 'no' # 'exponential' for var
mean_ann = ANN_online(n_in=n_in, n_out=n_out, hidden_channels=hidden_channels, ACTIVATION=activation, 
                      Xmean=old_model.Xscale['mean'], Xscale=old_model.Xscale['scale'], 
                      Ymean=old_model.Yscale['mean'], Yscale=old_model.Yscale['scale'])
# This will miss ['Xmean', 'Xscale', 'Ymean', 'Yscale'] because they were not registered as buffer in the old class
missing, unexpected = mean_ann.load_state_dict(old_model.mean_func.state_dict(), strict=False)
torch.save(mean_ann.state_dict(), dir_name + "mean_weights.pt")
print(missing)
print(unexpected)

####### Convert var network and save #######
n_in = 5
n_out = 1
hidden_channels = [32, 16] 
activation = 'exponential' # 'exponential' for var
var_ann = ANN_online(n_in=n_in, n_out=n_out, hidden_channels=hidden_channels, ACTIVATION=activation, 
                      Xmean=old_model.Xscale['mean'], Xscale=old_model.Xscale['scale'], 
                      Ymean=old_model.Yscale['mean'], Yscale=old_model.Yscale['scale'])
# This will miss ['Xmean', 'Xscale', 'Ymean', 'Yscale'] because they were not registered as buffer in the old class
missing, unexpected = var_ann.load_state_dict(old_model.var_func.state_dict(), strict=False)
torch.save(var_ann.state_dict(), dir_name + "var_weights.pt")

['Xmean', 'Xscale', 'Ymean', 'Yscale']
[]


### Test loading

In [7]:
from flux_model import ANN_online
import torch
target_path = '/scratch/jw8736/mlflux/example_torch_script/'
dir_name = target_path + 'LH/'
n_in = 5
n_out = 1
hidden_channels = [32, 16] 
activation = 'no' # 'exponential' for var
state_dict = torch.load(dir_name + "mean_weights.pt", map_location="cpu")
mean_ann = ANN_online(n_in=n_in, n_out=n_out, hidden_channels=hidden_channels, ACTIVATION=activation, 
                      Xmean=state_dict['Xmean'], Xscale=state_dict['Xscale'], Ymean=state_dict['Ymean'], Yscale=state_dict['Yscale'])
mean_ann.load_state_dict(state_dict)

<All keys matched successfully>

In [10]:
mean_ann.eval()
x = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0])
x = x.reshape(1, -1)
with torch.no_grad():
    y = mean_ann.forward(x)
print(y)

tensor([[-234.4499]])


### Check that it's consistent with old model

In [14]:
import os 
source_path = '/scratch/jw8736/mlflux/example_python/'
target_path = '/scratch/jw8736/mlflux/example_torch_script/'

# Apparently change M/SH/LH
old_model = open_case(source_path + 'LH/', 'lh.p') # this was a class not good for torch script
with torch.no_grad():
    y = old_model.pred_mean(x)
print(y)

tensor([[-234.4499]])


### Config dictionary

In [ ]:
###### Should I read from config 
# model_dir = '/scratch/jw8736/mlflux/example_python/M/'
# with open(model_dir + 'config.json', 'r') as f:
#     config = json.load(f)  
# hidden_channels = config['ann_size']
###### Or should I just hardcode it here
# ...

{'datapath': '/home/jw8736/mlflux/data/Processed/',
 'datafile': 'psd_coare3p0_weight1_wave0.nc',
 'ann_size': [32, 16],
 'mean_activation': 'no',
 'var_activation': 'exponential',
 'dropout_rate': -1,
 'ikeys': ['U', 'tsea', 'tair', 'rh', 'p'],
 'okeys': ['taucx'],
 'bkeys': ['taubx'],
 'RATIO': 0.2,
 'WEIGHT': False,
 'RESIDUAL': False,
 'training_paras': {'batchsize': 1000,
  'num_epochs': 4000,
  'lr': 0.0005,
  'gamma': 0.2,
  'EARLYSTOPPING': False,
  'patience': 20,
  'factor': 0.5,
  'max_epochs_without_improvement': 100,
  'weight_decay': 1e-05},
 'compute_norm': True,
 'two_steps': True,
 'VERBOSE': True}